In [ ]:
import requests
import pandas as pd
from io import StringIO

In [ ]:
def get_stock_data(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}"
    html_data = requests.get(url).text
    df = pd.read_html(html_data)[0]
    return df

In [ ]:
ticker = "AAPL"
def get_stock_data(ticker):    
    base_url = 'https://finance.yahoo.com'
    url = f"{base_url}/quote/{ticker}/key-statistics?p={ticker}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    data = requests.get(url=url, headers=headers).text
    if "Will be right back" in data.text:
        raise RuntimeError("*** YAHOO! FINANCE is currently down! ***\n")
    df = pd.read_html(StringIO(data))[0]
    print(df)
    return df

In [ ]:
def filter_value_stocks(df):
    pe_ratio = df.loc[df['Attribute'] == 'Trailing P/E']['Value'].iloc[0]
    pb_ratio = df.loc[df['Attribute'] == 'Price/Book']['Value'].iloc[0]
    if pe_ratio < 15 and pb_ratio < 1.5:
        return True
    else:
        return False

In [ ]:
tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'FB']

for ticker in tickers:
    df = get_stock_data(ticker)
    if filter_value_stocks(df):
        print(f"{ticker} is a value stock!")
    else:
        print(f"{ticker} is not a value stock.")

In [ ]:
import requests
import pandas as pd
from io import StringIO

PE_THRESHOLD = 15
PB_THRESHOLD = 1.5

def get_stock_data(ticker):    
    base_url = 'https://finance.yahoo.com'
    url = f"{base_url}/quote/{ticker}/key-statistics?p={ticker}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url=url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        
        if "Will be right back" in response.text:
            raise RuntimeError("*** YAHOO! FINANCE is currently down! ***\n")
        
        df = pd.read_html(StringIO(response.text))[0]
        df = df.rename(columns={'0': 'Metric', '1': 'Value'})
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None


def filter_value_stocks(df, pe_threshold=PE_THRESHOLD, pb_threshold=PB_THRESHOLD):
    pe_ratio = df.loc[df['0'] == 'Trailing P/E']['1'].iloc[0]
    pb_ratio = df.loc[df['0'] == 'Price/Book (mrq)']['1'].iloc[0]
    return pe_ratio < pe_threshold and pb_ratio < pb_threshold

tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'META']

for ticker in tickers:
    df = get_stock_data(ticker)
    print(df)
    if filter_value_stocks(df):
        print(f"{ticker} is a value stock! (P/E: {PE_THRESHOLD}, P/B: {PB_THRESHOLD})")
    else:
        print(f"{ticker} is not a value stock. (P/E: {PE_THRESHOLD}, P/B: {PB_THRESHOLD})")


In [ ]:
import requests
import pandas as pd
from io import StringIO

PE_THRESHOLD = 15
PB_THRESHOLD = 1.5


class YahooFinanceError(Exception):
    pass


def get_stock_data(ticker):
    base_url = "https://finance.yahoo.com"
    url = f"{base_url}/quote/{ticker}/key-statistics?p={ticker}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(url=url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

        if "Will be right back" in response.text:
            raise YahooFinanceError("*** YAHOO! FINANCE is currently down! ***\n")

        df = pd.read_html(StringIO(response.text))[0]
        df = df.rename(columns={0: "Metric", 1: "Value"})
        return df

    except requests.exceptions.RequestException as e:
        raise YahooFinanceError(f"Error fetching data for {ticker}: {e}")


def filter_value_stocks(df, pe_threshold=PE_THRESHOLD, pb_threshold=PB_THRESHOLD):
    if df is not None:
        pe_ratio = df.loc[df["Metric"] == "Trailing P/E"]["Value"].iloc[0]
        pb_ratio = df.loc[df["Metric"] == "Price/Book (mrq)"]["Value"].iloc[0]
        return pe_ratio < pe_threshold and pb_ratio < pb_threshold
    else:
        return False


tickers = ["AAPL", "GOOGL", "AMZN", "MSFT", "META"]

for ticker in tickers:
    try:
        df = get_stock_data(ticker)
        print(df)
        if filter_value_stocks(df):
            print(
                f"{ticker} is a value stock! (P/E: {PE_THRESHOLD}, P/B: {PB_THRESHOLD})"
            )
        else:
            print(
                f"{ticker} is not a value stock. (P/E: {PE_THRESHOLD}, P/B: {PB_THRESHOLD})"
            )
    except YahooFinanceError as e:
        print(e)

In [ ]:
import pandas as pd

# Assuming PE_THRESHOLD and PB_THRESHOLD are defined
PE_THRESHOLD = 15
PB_THRESHOLD = 1.5

# Create a sample DataFrame
data = {'0': ['Market Cap (intraday)', 'Enterprise Value', 'Trailing P/E', 'Forward P/E', 'PEG Ratio (5 yr expected)', 'Price/Sales (ttm)', 'Price/Book (mrq)', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA'],
        '1': ['2.95T', '3.00T', 30.99, 28.90, 2.25, 7.84, 47.54, 7.84, 23.25]}
df = pd.DataFrame(data)
df = df.rename(columns={'0': 'Metric', '1': 'Value'})

print(df)

def filter_value_stocks(df, pe_threshold=PE_THRESHOLD, pb_threshold=PB_THRESHOLD):
    pe_ratio = df.loc[df['0'] == 'Trailing P/E']['1'].iloc[0]
    pb_ratio = df.loc[df['0'] == 'Price/Book (mrq)']['1'].iloc[0]
    return pe_ratio < pe_threshold and pb_ratio < pb_threshold

# Check if a stock is a value stock
if filter_value_stocks(df):
    print("This stock is a value stock!")
else:
    print("This stock is not a value stock.")

In [ ]:
from yahooquery import Ticker
import pandas as pd

aapl = Ticker('aapl')
df = aapl.valuation_measures
df = df.ffill().bfill().drop_duplicates()
#df = df.dropna()
df = df.rename(columns={'asOfDate': 'Date'})
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime(
            "%Y-%m-%d"
        )
last_row_dict = df.iloc[-1].to_dict()
print(df)

In [ ]:
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_analysts_info
import pandas_datareader as pdr

data = {}
def get_data(ticker, ng_pe, multiplier, margin):
    quote = si.get_quote_table(ticker)
    current_price = quote["Quote Price"]
    eps = quote["EPS (TTM)"]
    growth_df = get_analysts_info(ticker)['Growth Estimates']
    growth_rate = growth_df.iloc[4][1]
    growth_rate = growth_rate.rstrip("%")
    aaa_df = pdr.get_data_fred('AAA')
    current_yield = aaa_df.iloc[-1][0]

    output = {
        "current_price": float(current_price),
        "eps": float(eps),
        "growth_rate": float(growth_rate),
        "current_yield": float(current_yield),
        "ng_pe": float(ng_pe),
        "multiplier": float(multiplier),
        "margin": float(margin)
    }
    print(output)
    return output

ticker = 'AAPL'
ng_pe = 8.5
multiplier = 2
margin = 35
data = get_data(ticker, ng_pe, multiplier, margin)
print(data)

In [ ]:
from yahooquery import Ticker

symbols = 'fb aapl amzn nflx goog'

tickers = Ticker(symbols)

# Retrieve each company's profile information
data = tickers.asset_profile
data

In [ ]:
pip install advanced-ta

In [ ]:
pip install TA-Lib

In [ ]:
from advanced_ta import LorentzianClassification
# Validate Lorentzian Classification signal  
def validateLorentzian(data, saveDict, lookFor=1):
    # lookFor: 1-Any, 2-Buy, 3-Sell
    data = data.rename(columns={'Open':'open', 'Close':'close', 'High':'high', 'Low':'low', 'Volume':'volume'})
    lc = LorentzianClassification(data=data)
    if lc.df.iloc[-1]['isNewBuySignal']:
        saveDict['Pattern'] = f'Lorentzian-Buy'
        if lookFor != 3:
            return saveDict
    elif lc.df.iloc[-1]['isNewSellSignal']:
        saveDict['Pattern'] = f'Lorentzian-Sell'
        if lookFor != 2:
            return saveDict
    return saveDict

In [9]:
import yfinance as yf
import pandas as pd
import datetime as dt

result = {}

msft = yf.Ticker("AAPL")

# get all stock info
print(msft.info)

df_hist = msft.history(period="3y",)
df_hist.index = df_hist.index.strftime('%Y-%m-%d')
df_hist['Adj Close'] = df_hist['Close'].copy()
df_hist= df_hist.reindex(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits'])
##result = validateLorentzian(df_hist, result)
#print(result)
msft.info

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [ ]:
from advanced_ta import LorentzianClassification
import TA-Lib as ta
import yfinance as yf

result = {}

msft = yf.Ticker("AAPL")
df_hist = msft.history(period="3y",)
df_hist.index = df_hist.index.strftime('%Y-%m-%d')
df_hist['Adj Close'] = df_hist['Close'].copy()
df = df_hist.reindex(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits'])

# df here is the dataframe containing stock data as [['open', 'high', 'low', 'close', 'volume']]. Notice that the column names are in lower case.
lc = LorentzianClassification(
    df,
    features=[
        LorentzianClassification.Feature("RSI", 14, 2),  # f1
        LorentzianClassification.Feature("WT", 10, 11),  # f2
        LorentzianClassification.Feature("CCI", 20, 2),  # f3
        LorentzianClassification.Feature("ADX", 20, 2),  # f4
        LorentzianClassification.Feature("RSI", 9, 2),   # f5
        ta.MFI(df['open'], df['high'], df['low'], df['close'], df['volume']) #f6
    ],
    settings=LorentzianClassification.Settings(
        source='close',
        neighborsCount=8,
        maxBarsBack=2000,
        useDynamicExits=False
    ),
    filterSettings=LorentzianClassification.FilterSettings(
        useVolatilityFilter=True,
        useRegimeFilter=True,
        useAdxFilter=False,
        regimeThreshold=-0.1,
        adxThreshold=20,
        kernelFilter = LorentzianClassification.KernelFilter(
            useKernelSmoothing = False
            lookbackWindow = 8
            relativeWeight = 8.0
            regressionLevel = 25
            crossoverLag = 2
        )
    ))
lc.dump('output/result.csv')
lc.plot('output/result.jpg')

In [1]:
pip install tradingview-ta

Note: you may need to restart the kernel to use updated packages.


In [5]:
from tradingview_ta import TA_Handler, Interval, Exchange

tesla = TA_Handler(
    symbol="TSLA",
    screener="america",
    exchange="NASDAQ",
    interval=Interval.INTERVAL_1_DAY,
)
print(tesla.get_analysis().summary['RECOMMENDATION'])
print(tesla.get_analysis().moving_averages['RECOMMENDATION'])
print(tesla.get_analysis().oscillators['RECOMMENDATION'])
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

STRONG_BUY
STRONG_BUY
NEUTRAL
